# Predict NYC Taxi Tips 
The notebook ingests, prepares and then trains a model based on an Open Dataset that tracks NYC Yellow Taxi trips and various attributes around them. The goal is to for a given trip, predict whether there will be a tip or not. The model then will be converted to ONNX format and tracked by MLFlow.
We will later use the ONNX model for inferencing in Azure Synapse SQL Pool using the new model scoring wizard.
## Note:
**Please note that for successful conversion to ONNX, this notebook requires using  Scikit-learn version 0.20.3.**
Run the first cell to list the packages installed and check your sklearn version. Uncomment the pip install command to install the correct version

%pip install scikit-learn==0.20.3



## Load data
Get a sample data of nyc yellow taxi from Azure Open Datasets

In [1]:
#%pip list
#%pip install scikit-learn==0.20.3

In [2]:
from azureml.opendatasets import NycTlcYellow
from datetime import datetime
from dateutil import parser

start_date = parser.parse('2018-05-01')
end_date = parser.parse('2018-05-07')
nyc_tlc = NycTlcYellow(start_date=start_date, end_date=end_date)
nyc_tlc_df = nyc_tlc.to_pandas_dataframe()
nyc_tlc_df.info()

[Info] read from /tmp/tmp8403w7ap/https/azureopendatastorage.azurefd.net/nyctlc/yellow/puYear=2018/puMonth=5/part-00000-tid-8898858832658823408-a1de80bd-eed3-4d11-b9d4-fa74bfbd47bc-426339-118.c000.snappy.parquet
[Info] read from /tmp/tmp8403w7ap/https/azureopendatastorage.azurefd.net/nyctlc/yellow/puYear=2018/puMonth=5/part-00001-tid-8898858832658823408-a1de80bd-eed3-4d11-b9d4-fa74bfbd47bc-426336-117.c000.snappy.parquet
[Info] read from /tmp/tmp8403w7ap/https/azureopendatastorage.azurefd.net/nyctlc/yellow/puYear=2018/puMonth=5/part-00002-tid-8898858832658823408-a1de80bd-eed3-4d11-b9d4-fa74bfbd47bc-426334-119.c000.snappy.parquet
[Info] read from /tmp/tmp8403w7ap/https/azureopendatastorage.azurefd.net/nyctlc/yellow/puYear=2018/puMonth=5/part-00003-tid-8898858832658823408-a1de80bd-eed3-4d11-b9d4-fa74bfbd47bc-426340-115.c000.snappy.parquet
[Info] read from /tmp/tmp8403w7ap/https/azureopendatastorage.azurefd.net/nyctlc/yellow/puYear=2018/puMonth=5/part-00004-tid-8898858832658823408-a1de80bd

In [3]:
from IPython.display import display

sampled_df = nyc_tlc_df.sample(n=10000, random_state=123)
display(sampled_df.head(5))

vendorID  tpepPickupDateTime  ... tollsAmount  totalAmount
87213         2 2018-05-05 19:07:01  ...        5.76        23.56
145405        2 2018-05-05 22:46:06  ...        0.00        12.96
457648        1 2018-05-06 18:53:06  ...        0.00        11.80
369051        2 2018-05-02 09:25:13  ...        0.00        10.56
38871         2 2018-05-04 02:58:10  ...        0.00         6.62

[5 rows x 21 columns]

## Prepare and featurize data
- There are extra dimensions that are not going to be useful in the model. We just take the dimensions that we need and put them into the featurised dataframe. 
- There are also a bunch of outliers in the data so we need to filter them out.

In [4]:
import numpy
import pandas

def get_pickup_time(df):
    pickupHour = df['pickupHour'];
    if ((pickupHour >= 7) & (pickupHour <= 10)):
        return 'AMRush'
    elif ((pickupHour >= 11) & (pickupHour <= 15)):
        return 'Afternoon'
    elif ((pickupHour >= 16) & (pickupHour <= 19)):
        return 'PMRush'
    else:
        return 'Night'

featurized_df = pandas.DataFrame()
featurized_df['tipped'] = (sampled_df['tipAmount'] > 0).astype('int')
featurized_df['fareAmount'] = sampled_df['fareAmount'].astype('float32')
featurized_df['paymentType'] = sampled_df['paymentType'].astype('int')
featurized_df['passengerCount'] = sampled_df['passengerCount'].astype('int')
featurized_df['tripDistance'] = sampled_df['tripDistance'].astype('float32')
featurized_df['pickupHour'] = sampled_df['tpepPickupDateTime'].dt.hour.astype('int')
featurized_df['tripTimeSecs'] = ((sampled_df['tpepDropoffDateTime'] - sampled_df['tpepPickupDateTime']) / numpy.timedelta64(1, 's')).astype('int')

featurized_df['pickupTimeBin'] = featurized_df.apply(get_pickup_time, axis=1)
featurized_df = featurized_df.drop(columns='pickupHour')

display(featurized_df.head(5))


tipped  fareAmount  ...  tripTimeSecs  pickupTimeBin
87213        0        17.0  ...          1303         PMRush
145405       1         9.5  ...           785          Night
457648       0        11.0  ...           805         PMRush
369051       1         8.0  ...           679         AMRush
38871        1         4.0  ...           180          Night

[5 rows x 7 columns]

In [5]:
filtered_df = featurized_df[(featurized_df.tipped >= 0) & (featurized_df.tipped <= 1)\
    & (featurized_df.fareAmount >= 1) & (featurized_df.fareAmount <= 250)\
    & (featurized_df.paymentType >= 1) & (featurized_df.paymentType <= 2)\
    & (featurized_df.passengerCount > 0) & (featurized_df.passengerCount < 8)\
    & (featurized_df.tripDistance >= 0) & (featurized_df.tripDistance <= 100)\
    & (featurized_df.tripTimeSecs >= 30) & (featurized_df.tripTimeSecs <= 7200)]

filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9776 entries, 87213 to 333274
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tipped          9776 non-null   int64  
 1   fareAmount      9776 non-null   float32
 2   paymentType     9776 non-null   int64  
 3   passengerCount  9776 non-null   int64  
 4   tripDistance    9776 non-null   float32
 5   tripTimeSecs    9776 non-null   int64  
 6   pickupTimeBin   9776 non-null   object 
dtypes: float32(2), int64(4), object(1)
memory usage: 534.6+ KB

## Split training and testing data sets
- 70% of the data is used to train the model.
- 30% of the data is used to test the model.

In [6]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(filtered_df, test_size=0.3, random_state=123)

x_train = pandas.DataFrame(train_df.drop(['tipped'], axis = 1))
y_train = pandas.DataFrame(train_df.iloc[:,train_df.columns.tolist().index('tipped')])

x_test = pandas.DataFrame(test_df.drop(['tipped'], axis = 1))
y_test = pandas.DataFrame(test_df.iloc[:,test_df.columns.tolist().index('tipped')])

## Export test data as CSV
Export the test data as a CSV file. Later, we will load the CSV file into Synapse SQL pool to test the model.

In [7]:
test_df.to_csv('test_data.csv', index=False)

## Train model
Train a bi-classifier to predict whether a taxi trip will be a tipped or not.

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

float_features = ['fareAmount', 'tripDistance']
float_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

integer_features = ['paymentType', 'passengerCount', 'tripTimeSecs']
integer_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['pickupTimeBin']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('float', float_transformer, float_features),
        ('integer', integer_transformer, integer_features),
        ('cat', categorical_transformer, categorical_features)
    ])

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(solver='lbfgs'))])

# Train the model
clf.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('float', Pipeline(memory=None,
     steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='median', ver...enalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))])
/home/trusted-service-user/cluster-env/env/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

In [9]:
# Evalute the model
score = clf.score(x_test, y_test)
print(score)

0.9672690078418003

## Convert the model to ONNX format
Currently, T-SQL scoring only supports ONNX model format (https://onnx.ai/).

In [10]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType, Int64TensorType, DoubleTensorType, StringTensorType

def convert_dataframe_schema(df, drop=None):
    inputs = []
    for k, v in zip(df.columns, df.dtypes):
        if drop is not None and k in drop:
            continue
        if v == 'int64':
            t = Int64TensorType([1, 1])
        elif v == 'float32':
            t = FloatTensorType([1, 1])
        elif v == 'float64':
            t = DoubleTensorType([1, 1])
        else:
            t = StringTensorType([1, 1])
        inputs.append((k, t))
    return inputs

model_inputs = convert_dataframe_schema(x_train)
onnx_model = convert_sklearn(clf, "nyc_taxi_tip_predict", model_inputs)

## Register the model with MLFlow

In [11]:
# Azure MLと接続
from azureml.core import Workspace

ws = Workspace.get(name="mlverify-mlws",
            subscription_id='a1d6eb75-4814-4275-8647-9addeb8fe6e3',
            resource_group='BAP-ZEALWorkshop-RG-01-Prd')

In [12]:
import mlflow
import mlflow.onnx

from mlflow.models.signature import infer_signature

experiment_name = 'nyc_taxi_tip_predict_exp'
artifact_path = 'nyc_taxi_tip_predict_artifact'

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
mlflow.set_experiment(experiment_name)

with mlflow.start_run() as run:
    # Infer signature
    input_sample = x_train.head(1)
    output_sample = pandas.DataFrame(columns=['output_label'], data=[1])
    signature = infer_signature(input_sample, output_sample)

    # Save the model to the outputs directory for capture
    mlflow.onnx.log_model(onnx_model, artifact_path, signature=signature, input_example=input_sample)

    # Register the model to AML model registry
    mlflow.register_model('runs:/' + run.info.run_id + '/' + artifact_path, 'nyc_taxi_tip_predict')


INFO: 'nyc_taxi_tip_predict_exp' does not exist. Creating a new experiment
<ModelVersion: creation_timestamp=1603187804853, current_stage='None', description='', last_updated_timestamp=1603187804853, name='nyc_taxi_tip_predict', run_id='a415d769-1258-4d6f-a7ec-e076cce93cc8', run_link='', source='azureml://experiments/nyc_taxi_tip_predict_exp/runs/a415d769-1258-4d6f-a7ec-e076cce93cc8/artifacts/nyc_taxi_tip_predict_artifact', status='READY', status_message='', tags={}, user_id='', version='1'>
2020/10/20 09:56:43 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under azureml://experiments/nyc_taxi_tip_predict_exp/runs/a415d769-1258-4d6f-a7ec-e076cce93cc8/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server vers